In [3]:
import os

import pandas as pd
"""버스전용차로 분석 데이터"""
# 1. CSV 파일 불러오기
file_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\버스전용차로\분석데이터\기흥IC~신갈JC_차간간격_50-50_001\기흥IC~신갈JC_차간간격_50-50_001.fzp"
file_name = os.path.splitext(os.path.basename(file_path))[0]

df = pd.read_csv(file_path, sep=";" ,low_memory=False)

# 2. 필요한 컬럼만 추출 (메모리 최적화)
df = df[["NO", "VEHTYPE", "LANE\\LINK\\NO", "LANE\\INDEX", "$VEHICLE:SIMSEC", "POS", "SPEED"]]

# 3. A 차량 필터링: 링크번호 18번, 5차로, 버스
a_df = df[(df["LANE\\LINK\\NO"] == 18) & (df["LANE\\INDEX"] == 5) & (df["VEHTYPE"] == 300)].copy()

# 4. B 차량 후보: 4차로만 필터링 + 시간 기준으로 그룹화
df_4th_lane = df[df["LANE\\INDEX"] == 4].copy()
df_4th_lane_grouped = df_4th_lane.groupby("$VEHICLE:SIMSEC")

# 5. 결과 저장 리스트
results = []

# 6. 샘플로 A 차량 일부만 선택 (전체 적용 시 a_df로 변경 가능)
a_sample = a_df

# 7. A 차량 기준으로 가장 가까운 B + B의 전방 C 추출
for _, a in a_sample.iterrows():
    simsec = a["$VEHICLE:SIMSEC"]
    a_pos = a["POS"]

    if simsec not in df_4th_lane_grouped.groups:
        continue

    b_set = df_4th_lane_grouped.get_group(simsec)
    if b_set.empty:
        continue

    # A와 위치 차이가 5 이내인 B 후보만 선택
    b_candidates = b_set[(b_set["POS"] - a_pos).abs() <= 5]

    if not b_candidates.empty:
        # 그 중 가장 가까운 B 선택
        b_row = b_candidates.loc[(b_candidates["POS"] - a_pos).abs().idxmin()]
    else:
        continue  # 조건에 맞는 B가 없으면 스킵

    # B 차량의 전방 C 차량
    front_candidates = b_set[b_set["POS"] > b_row["POS"]]
    if not front_candidates.empty:
        c_row = front_candidates.loc[front_candidates["POS"].idxmin()]
        results.append({
            "A_NO": a["NO"], "A_POS": a["POS"], "A_SPEED": a["SPEED"],
            "B_NO": b_row["NO"], "B_POS": b_row["POS"], "B_SPEED": b_row["SPEED"],
            "C_NO": c_row["NO"], "C_POS": c_row["POS"], "C_SPEED": c_row["SPEED"],
            "SIMSEC": simsec
        })
    else:
        results.append({
            "A_NO": a["NO"], "A_POS": a["POS"], "A_SPEED": a["SPEED"],
            "B_NO": b_row["NO"], "B_POS": b_row["POS"], "B_SPEED": b_row["SPEED"],
            "C_NO": None, "C_POS": None, "C_SPEED": None,
            "SIMSEC": simsec
        })

# 8. 결과 DataFrame 생성
result_df = pd.DataFrame(results)

# 원하는 컬럼 순서 지정
new_order = ['SIMSEC', 'A_NO', 'B_NO', 'C_NO', 'A_POS', 'B_POS', 'C_POS', 'A_SPEED', 'B_SPEED', 'C_SPEED']

# 컬럼 순서 재정렬
result_df = result_df[new_order]



display(result_df)
# 결과 확인 (예: result_df.head())


excel_file_path = fr"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\버스전용차로\분석데이터\기흥IC~신갈JC_차간간격_50-50_001\{file_name}_result2.xlsx"
result_df.to_excel(excel_file_path, index=True)


,SIMSEC,A_NO,B_NO,C_NO,A_POS,B_POS,C_POS,A_SPEED,B_SPEED,C_SPEED
0,901.0,616.0,576.0,572.0,6872.181,6876.982,6896.528,50.88,48.03,48.45
1,901.0,659.0,651.0,626.0,6394.049,6395.642,6411.999,49.73,47.02,47.99
2,901.0,706.0,739.0,735.0,5731.974,5728.719,5746.098,49.70,49.88,48.87
3,901.0,709.0,745.0,739.0,5706.175,5704.861,5728.719,48.56,52.00,49.88
4,901.0,785.0,780.0,779.0,5120.964,5123.121,5163.194,49.44,48.83,45.76
...,...,...,...,...,...,...,...,...,...,...
52049,4500.0,6841.0,6839.0,6835.0,1581.829,1581.459,1600.625,47.23,48.52,48.00
52050,4500.0,6853.0,6855.0,6848.0,1433.328,1432.114,1461.261,49.83,51.08,53.55
52051,4500.0,6864.0,6860.0,6855.0,1411.247,1413.628,1432.114,47.84,47.50,51.08
52052,4500.0,6945.0,6948.0,6083.0,543.908,544.160,547.777,49.54,49.69,51.54
